In [1]:
import os
import sys
import re
sys.path.append("../")

from utils.io import read_csv, load_json, write_to

In [2]:
def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248

        rstring += chr(inside_code)
    return rstring

def get_sighan_from_json():

    all_data = {
        "train":None,
        "dev":None,
        "test":None,
        "test14":None,
        "test15":None,
    }
    data_dir = "../data/rawdata/sighan/csc/"

    train_file1 = os.path.join(data_dir, "train_dev.json")
    train_file2 = os.path.join(data_dir, "train131415.json") 
    test14_file = os.path.join(data_dir, "test14.json")
    test15_file = os.path.join(data_dir, "test15.json")

    all_data["train"] = load_json(train_file1)
    all_data["train"].extend(load_json(train_file2))

    all_data["train"] = all_data["train"]

    all_data["valid14"] = load_json(test14_file)
    all_data["valid"] = load_json(test15_file)
    #all_data["test"].extend(load_json(test15_file))

    return all_data

def json2list(data, need_preprocess):
    source, target, ids = [], [], []

    for element in data:

        if need_preprocess:
            source.append(element["original_text"])
            target.append(element["correct_text"])
            ids.apoend(element["wrong_ids"])
        else:
            source.append(strQ2B((element["original_text"])))
            target.append(strQ2B((element["correct_text"])))
            ids.append(element["wrong_ids"])

    return source, target, ids

In [4]:
data = get_sighan_from_json()

source, target, ids = json2list(data["train"], False)
#source, target, ids = json2list(data["valid"], False)

raw_bert_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_base_std.epoch10.bs128/generated_predictions.txt")
#raw_bert_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_base_raw.epoch10.bs128/generated_predictions.txt")

#proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bshert_Flat_std_char.epoch10.bs16/generated_predictions.txt")
proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_v2_std.epoch10.bs32/generated_predictions.txt")
#proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_v2_std_mask.epoch20.bs32/generated_predictions.txt")
#proto_preds = read_csv("./no_mask_preds.txt")
#proto_preds = read_csv("./no_mask_preds_super.txt")
#proto_preds = read_csv("./no_mask_preds_overfit.txt")
#proto_preds = read_csv("./no_mask_bert_preds_overfit.txt")


for i in range(len(source[:1100])):
    #raw_bert_preds[i] = "".join(raw_bert_preds[i].split())[:len(source[i])]
    proto_preds[i] = "".join(proto_preds[i].split())[:len(source[i])] 

In [4]:
print(len(source), len(raw_bert_preds), len(target), len(proto_preds))

1100 1100 1100 1100


In [5]:
write_to("./newest_preds.txt", "\n".join(proto_preds))

In [5]:
need_cor = []
no_need_cor = []

def get_cor(source, target):
    return [ (j, target[j]) for j in range(len(target)) if target[j] != source[j]]


truth_cor_host, bert_cor_host, proto_cor_host = [], [], [] 

for i in range(len(source)):
    truth_cor, bert_cor, proto_cor = get_cor(source[i], target[i]), get_cor(source[i], raw_bert_preds[i]), get_cor(source[i], proto_preds[i])
    
    truth_cor_host.append(truth_cor)
    bert_cor_host.append(bert_cor)
    proto_cor_host.append(proto_cor)
    

In [6]:
print(truth_cor_host[0:10], bert_cor_host[:10])

[[], [(9, '友')], [], [(2, '起')], [(14, '舞')], [], [], [(3, '餐')], [(0, '走')], []] [[], [(9, '友')], [], [(2, '起')], [(14, '舞')], [], [], [(3, '餐')], [(0, '走')], []]


In [7]:
#check if lexcions perform
yjzheng_result = ['小鸡', '记得', '糊涂', '订位', '怎么', '面见', '弟弟', '炒饭', '汉字', '那店', '八点钟', '这样', '迟到', '很饱', '迟到', '迟到', '安静', '一点', '妈妈', '吃饭', '时候', '希望', '简讯', '那里', '诉取', '不半工', '哪些', '上半', '上半', '轻松', '消息', '与此同时', '这种', '权利', '优家', '失业', '心情', '减少', '著迷', '家长', '唠叨', '影响', '睡觉', '拜托', '存到', '年轻', '威胁', '我们', '回覆', '墙壁', '他们', '这是', '现在', '很忙', '不怎么', '试试看', '录影机', '哪个', '哪个', '不是', '再加', '该不该', '看著', '看著', '有装', '教书', '记录']
print(len(yjzheng_result))
print(source[0])

yjzheng_ids = []
for i in range(len(source)):
    for j in yjzheng_result:
        if re.search(j, target[i]) and (not re.search(j, source[i])) and (not re.search(j, raw_bert_preds[i])) and (re.search(j, proto_preds[i])):
            print(j)
            yjzheng_ids.append(i)

print(len(yjzheng_ids))
print(yjzheng_ids)

67
你好!我是张爱文。
怎么
面见
汉字
心情
唠叨
拜托
存到
不是
看著
看著
10
[83, 90, 164, 700, 807, 833, 838, 1060, 1085, 1085]


In [8]:
#样本条数 proto比bert好的

proto_better_than_bert = 0

proto_worse_than_bert = 0

def isequal(truth_cor, now_cor):
    #cor: [(index, char), ...]
    #if equal
    if len(truth_cor) != len(now_cor):
        return False

    for i in range(len(truth_cor)):
        if truth_cor[i] != now_cor[i]:
            return False
    return True

better_list = []
worse_list = []

for i in range(len(truth_cor_host)):
    if isequal(proto_cor_host[i], truth_cor_host[i]) and (not isequal(bert_cor_host[i], truth_cor_host[i])):
        proto_better_than_bert += 1
        better_list.append(i)
        #print(source[i])
    elif (not isequal(proto_cor_host[i], truth_cor_host[i])) and (isequal(bert_cor_host[i], truth_cor_host[i])):
        proto_worse_than_bert += 1
        worse_list.append(i)
    #break
print("better: ", proto_better_than_bert)
print("worse: ", proto_worse_than_bert)

better:  61
worse:  56


In [9]:
def short_show(name, ids_list, source, truth_cor_host, bert_cor_host, proto_cor_host):
    tmp = ""

    for i in ids_list:
        first = source[i] + "\n"
        second = ["  "] * len(source[i]) + ["\n"]
        for cor in truth_cor_host[i]:
            second[cor[0]] = cor[1]

        third = ["  "] * len(source[i]) + ["\n"]
        for cor in bert_cor_host[i]:
            third[cor[0]] = cor[1]

        fourth = ["  "] * len(source[i]) + ["\n"]
        for cor in proto_cor_host[i]:
            fourth[cor[0]] = cor[1]
        
        tmp += "".join(first) + "".join(second) + "".join(third) + "".join(fourth) + "\n"

    write_to("tmp_analysis_" + name + ".txt", tmp)

    return


In [14]:
short_show("better_v4", better_list, source ,truth_cor_host, bert_cor_host, proto_cor_host)

short_show("worse_v4", worse_list, source, truth_cor_host, bert_cor_host, proto_cor_host)


In [9]:
def compute_metrics_visuial(sources, preds, labels):
        """
        """

        tp, fp, fn = 0, 0, 0

        sent_p, sent_n = 0, 0

        for i in range(len(sources)):
            
            source, pred, label = sources[i], preds[i], labels[i]
            
            source = source[:len(label)]
            pred = pred[:len(label)]

            if not (pred == source):
                sent_p += 1
                if (pred == label):
                    tp += 1

            if (label != source):
                sent_n += 1

        precision = tp / sent_p
        recall = tp / sent_n
        F1_score = 2 * precision * recall / (precision + recall + 1e-10)

        print("precision: ", precision, "recall: ", recall, "F1_score: ", F1_score)

In [12]:
compute_metrics_visuial(source, raw_bert_preds, target)

precision:  0.7040816326530612 recall:  0.7638376383763837 F1_score:  0.7327433627819413


In [10]:
compute_metrics_visuial(source[:1100], proto_preds, target[:1100])

precision:  0.0 recall:  0.0 F1_score:  0.0


In [15]:
joint = ""

for i in range(len(source)):
    first = source[i] + "\n"
    second = ["  "] * len(source[i]) + ["\n"]
    if truth_cor_host[i]:
        for cor in truth_cor_host[i]:
            second[cor[0]] = cor[1]
    
    third = ["  "] * len(source[i]) + ["\n"]
    if bert_cor_host[i]:
        for cor in bert_cor_host[i]:
            third[cor[0]] = cor[1]

    fourth = ["  "] * len(source[i]) + ["\n"]
    if proto_cor_host[i]:
        for cor in proto_cor_host[i]:
            fourth[cor[0]] = cor[1]
    joint += "".join(first) + "".join(second) + "".join(third) + "".join(fourth) + "\n"


write_to("tmp_analysis_bert_MLM_v4_mask.txt", joint)
